# Instruções para o uso do Notebook

Para rodar o notebook a seguir corretamente, é necessário seguir alguns passos prévios. Primeiro, baixe os datasets disponíveis no link a seguir: [Datasets](https://drive.google.com/drive/folders/1bm6iQenyZ63gbw_s7j0md8UaHP84O0Vn?usp=sharing) (não é necessário todos, a princípio, somente os citados abaixo). Esses datasets não estão incluídos no notebook devido ao seu tamanho, por isso são ignorados pelo arquivo `.gitignore`. Todos eles devem ser colocados na pasta [`data`](data), localizada em `src/Notebook/Api_Sprint_4/data`.

Os arquivos necessários são:

- **GoogleNews-vectors-negative300.bin**: [Download](https://drive.google.com/open?id=1IzDDngIEWzGP4onWlBCglAQ87g5gP2CL&usp=drive_copy) - contém palavras pré-treinadas em 300 dimensões.
- **classification-labeled.csv**: [Download](https://drive.google.com/file/d/1AjRl3mKWceHjTFEZsHUrAwCRs-12UQQj/view?usp=drive_link) - contém o banco de dados original, entregue pela *Uber*.
- **processed_text_data.csv**: [Download](https://drive.google.com/file/d/1hCW3pliSKDWKyr0h0tvNBtUgtw9EsE3c/view?usp=sharing) - contém o banco de dados processado.

Todos são necessários para rodar as células seguintes.

Outras recomendações:

- Mantenha o C++ do seu computador atualizado, pois determinadas bibliotecas são compiladas com essa linguagem.
- Caso queira conferir as versões mais atualizaas das bibliotecas, elas estão detalhadas no aquivo [``requirements.txt``](../../requirements.txt). Para usar este arquivo, basta rodar no terminal o comando: ``pip install -r requirements.txt``. Porém, a princípio, é possível ter acesso a todas as bilioteca ao rodar a célula de "Instalação e Importação das Bibliotecas" deste Notebook.

# Instalação e Importação das Bibliotecas

Na seção "Instalação e Importação das Bibliotecas", prepara-se o ambiente de codificação configurando e carregando as bibliotecas necessárias para o projeto. Esta seção é importante pelos seguintes motivos:

1. **Instalação de Bibliotecas**: Utilizam-se comandos como `%pip install nome_da_biblioteca` para instalar bibliotecas Python que não estão presentes por padrão no ambiente de execução, mas que são essenciais para a execução do código. Este comando mágico oferece uma integração mais direta com o IPython, garantindo que as instalações ocorram no kernel correto do Python utilizado pelo Jupyter.

2. **Importação de Bibliotecas**: Após a instalação, as bibliotecas são importadas usando o comando `import`, permitindo o acesso às funções e ferramentas disponibilizadas por elas. Comandos típicos incluem `import numpy as np` e `import pandas as pd`. Esta prática garante que todas as funcionalidades necessárias estejam disponíveis e prontas para uso nas seções subsequentes do notebook.

Esta seção é posicionada no início do notebook para assegurar que todas as dependências estejam corretamente configuradas antes de prosseguir com a análise de dados ou modelagem.

In [ ]:
# Instalação de Bibliotecas Necessárias
%pip install pandas
%pip install numpy
%pip install nltk
%pip install emot
%pip install matplotlib
%pip install scikit-learn
%pip install gensim
%pip install seaborn
%pip install xgboost
%pip install scipy==1.11

!python -m spacy download en_core_web_md

In [ ]:
# Importação de Bibliotecas
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from gensim.models import KeyedVectors
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from emot.emo_unicode import UNICODE_EMOJI  
import xgboost as xgb
import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os

# Pré-Processamento dos dados


Abaixo, foram realizadas as seguintes etapas de pré-processamento:
- Conversão de emojis para palavras;
- Remoção de URLs, tags HTML, caracteres especiais e números;
- Tokenização;
- Remoção de Stop-Words;
- Stemming e lematização.

In [ ]:
# Carregando os dados
classification_labeled = pd.read_csv('./data/classification-labeled.csv', delimiter=';', encoding='ISO-8859-1')

In [ ]:
def convert_emojis(text):
    '''
    Converte emojis no texto para palavras.
    Input: texto original como uma string.
    Output: texto com emojis convertidos como uma string.
    '''
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",", "").replace(":", "").split()))
    return text

# Aplicando a função convert_emojis e exibindo os resultados
classification_labeled['converted_emojis'] = classification_labeled['comment'].apply(convert_emojis)

def clean_text(text):
    '''
    Limpa o texto removendo URLs, tags HTML, caracteres especiais e números através de expressões regulares.

    Input: texto original como uma string.
    Output: texto limpo, sem URLs, tags HTML, caracteres especiais e números, como uma string.
    '''
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'<[^>]+>', '', text)  # Remove tags HTML
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove caracteres especiais e números
    text = re.sub(r'\s+', ' ', text).strip() # Corrigir espaços extras

    return text

# Aplicando a função clean_text e exibindo os resultados
classification_labeled['cleaned_text'] = classification_labeled['comment'].apply(clean_text)

def tokenize(text):
    '''
    Tokeniza o texto em palavras.
    
    Input: texto limpo como uma string.
    Output: lista de palavras (tokens).
    '''
    tokens = word_tokenize(text.lower())  # Convertendo para minúsculas e tokenizando
    return tokens

# Aplicando a função tokenize e exibindo os resultados
classification_labeled['tokenized_text'] = classification_labeled['comment'].apply(tokenize)

def remove_stopwords(tokens):
    '''
    Remove stopwords do texto tokenizado.

    Input: lista de palavras (tokens).
    Output: lista de palavras sem stopwords.
    '''
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens

# Aplicando a função remove_stopwords e exibindo os resultados
classification_labeled['without_stopwords'] = classification_labeled['comment'].apply(tokenize).apply(remove_stopwords)

def stem_and_lemmatize(tokens):
    '''
    Aplica stemming e lematização nas palavras para reduzir à forma base.

    Input: lista de palavras (tokens).
    Output: lista de palavras processadas.
    '''
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    processed_tokens = [lemmatizer.lemmatize(stemmer.stem(token)) for token in tokens]
    return processed_tokens

# Aplicando a função stem_and_lemmatize e exibindo os resultados
classification_labeled['stemmed_and_lemmatized'] = classification_labeled['comment'].apply(tokenize).apply(remove_stopwords).apply(stem_and_lemmatize)

def preprocess_pipeline(dataframe, text_column):
    '''
    Pipeline de pré-processamento que aplica todas as funções acima ao texto.

    Input: DataFrame e o nome da coluna de texto.
    Output: DataFrame com o texto pré-processado.
    '''
    '''Aplicando funções de pré-processamento'''
    dataframe['processed_text'] = dataframe[text_column].apply(lambda x: convert_emojis(x))
    dataframe['processed_text'] = dataframe['processed_text'].apply(lambda x: clean_text(x))
    dataframe['processed_text'] = dataframe['processed_text'].apply(lambda x: tokenize(x))
    dataframe['processed_text'] = dataframe['processed_text'].apply(lambda x: remove_stopwords(x))
    dataframe['processed_text'] = dataframe['processed_text'].apply(lambda x: stem_and_lemmatize(x))
    dataframe['processed_text'] = dataframe['processed_text'].apply(lambda x: ' '.join(x))
    return dataframe

# Carregando os dados
classification_labeled = pd.read_csv('./data/classification-labeled.csv', delimiter=';', encoding='ISO-8859-1')

# Executando o pipeline de pré-processamento
processed_df = preprocess_pipeline(classification_labeled, 'comment')

# Salvando os dados processados em um novo arquivo CSV
processed_df.to_csv('processed_text_data.csv', index=False)

processed_df

## Bag of Words

Nesta seção, implementa-se o modelo Bag of Words nos dados pré-processados. A aplicação do Bag of Words aos dados pré-processados permite uma análise mais estruturada e quantitativa, facilitando a aplicação de técnicas de aprendizado de máquina para tarefas como classificação e análise de sentimentos.

In [ ]:
processed_text_data = pd.read_csv('./data/processed_text_data.csv')
processed_text_data.head()

In [ ]:
# A seguir é realizado o modelo Bag of Words nos dados pré-processados

def generate_bow(dataframe, text_column):
    """
    Gera um DataFrame Bag of Words a partir de uma coluna de texto especificada de um DataFrame.

    input:
    dataframe (pd.DataFrame): DataFrame contendo a coluna de texto.
    text_column (str): Nome da coluna de texto que será processada para criar o Bag of Words.

    output:
    pd.DataFrame: DataFrame representando o Bag of Words com cada palavra como uma coluna e cada documento como uma linha.
    """
    # Inicializa o vetorizador de contagem
    vectorizer = CountVectorizer()

    # Ajusta e transforma os dados da coluna de texto especificada
    X = vectorizer.fit_transform(dataframe[text_column])

    # Obtém o vocabulário
    vocab = vectorizer.get_feature_names_out()

    # Cria um DataFrame a partir da matriz esparsa, com o vocabulário como colunas
    bow_data = pd.DataFrame(X.toarray(), columns=vocab)

    return bow_data


# Aplicar a função ao DataFrame
bow_data = generate_bow(processed_text_data, 'processed_text')

# Exibir o DataFrame resultante
print(bow_data)

# Word2Vec

Essa seção contém o uso de Word2Vec de 300 dimensões, utilizando o conjunto de vetores pré-treinados GoogleNews em inglês, especificamente o arquivo `GoogleNews-vectors-negative300.bin`, que pode ser acessado [nesse link](https://drive.google.com/file/d/1IzDDngIEWzGP4onWlBCglAQ87g5gP2CL/view?usp=drive_link).

In [ ]:
EMBEDDING_FILE = './data/GoogleNews-vectors-negative300.bin'
word_vectors = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
def prepare_word2vec_vectors(csv_file_path, embedding_file):
    """
    Carrega dados de um arquivo CSV e utiliza um modelo Word2Vec pré-treinado para
    gerar representações vetoriais dos textos.

    Inputs:
        csv_file_path (str): Caminho para o arquivo CSV contendo a coluna de texto.
        embedding_file (str): Caminho para o arquivo binário do modelo Word2Vec pré-treinado.

    Output: DataFrame: DataFrame com os vetores de palavras para cada documento.

    """
    # Carregar o modelo pré-treinado
    word_vectors = KeyedVectors.load_word2vec_format(embedding_file, binary=True)

    # Ler o arquivo CSV
    data = pd.read_csv(csv_file_path)

    # Assume-se que a coluna com texto processado é chamada 'processed_text'
    texts = data['processed_text'].astype(str)

    # Inicializar um vetorizador para tokenizar os textos
    vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')  # Tokeniza palavras
    vectorizer.fit(texts)

    # Inicializar matriz para armazenar vetores de palavras
    word_embeddings = np.zeros((len(texts), word_vectors.vector_size))

    for i, text in enumerate(texts):
        tokens = vectorizer.build_tokenizer()(text)
        word_vecs = [word_vectors[word] for word in tokens if word in word_vectors.key_to_index]
        if word_vecs:
            word_embeddings[i] = np.sum(word_vecs, axis=0)  

    # Criar um DataFrame com os vetores
    return pd.DataFrame(word_embeddings)

In [ ]:
# Chamada da Função
embedding_file = './data/GoogleNews-vectors-negative300.bin'
csv_file_path = './data/processed_text_data.csv'
vector_df = prepare_word2vec_vectors(csv_file_path, embedding_file)

In [ ]:
# Transforma em arquivo csv
vector_df.to_csv('./data/word_2_vec_text_data_300.csv', index=False)
vector_df.head()

In [ ]:
def merge_sentiment_column(classification_csv, word2vec_csv, output_csv):
    """
    Carrega dois arquivos CSV, adiciona a coluna 'sentiment' do primeiro arquivo
    ao segundo arquivo, e salva o resultado em um novo arquivo CSV.

    Args:
    classification_csv (str): Caminho para o arquivo CSV contendo a coluna 'sentiment'.
    word2vec_csv (str): Caminho para o arquivo CSV contendo os vetores de palavras.
    output_csv (str): Caminho para o arquivo CSV onde o resultado será salvo.
    
    Returns:
    None
    """
    # Carregar os arquivos CSV
    classification_text_data = pd.read_csv(classification_csv)
    word_2_vec_data = pd.read_csv(word2vec_csv)

    # Selecionar a coluna 'sentiment' do arquivo classification_text_data
    sentiment_column = classification_text_data['sentiment']

    # Adicionar a coluna 'sentiment' no arquivo word_2_vec_data na primeira coluna (coluna 0)
    word_2_vec_data.insert(0, 'sentiment', sentiment_column)

    # Verificar o resultado
    print(word_2_vec_data.head())

    # Salvar o novo DataFrame em um novo arquivo CSV
    word_2_vec_data.to_csv(output_csv, index=False)

In [ ]:
# Chamando a função que 1: vetoriza o banco de dados utilizando o Word2Vec e 2: cria o dataset para treinar os modelos 
merge_sentiment_column('./data/processed_text_data.csv', './data/word_2_vec_text_data_300.csv', './data/data_for_training_models.csv')

# TD-IDF

TF-IDF (Term Frequency-Inverse Document Frequency) é uma técnica amplamente utilizada na recuperação de informações e mineração de texto para refletir a importância de uma palavra em um documento em relação a um corpus (conjunto de documentos).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the uploaded CSV file
file_path = 'data/processed_text_data.csv'
data = pd.read_csv(file_path)

def compute_tfidf(dataframe, text_column):
    """
    Calcula a matriz TF-IDF para um DataFrame e coluna de texto fornecidos.

    Input:
    dataframe (pd.DataFrame): O DataFrame de entrada contendo os dados de texto.
    text_column (str): O nome da coluna contendo o texto processado.

    Output:
    pd.DataFrame: Um DataFrame contendo a matriz TF-IDF.
    """
    # Cria um Vetorizador TF-IDF
    tfidf_vectorizer = TfidfVectorizer()

    # Ajusta e transforma o texto processado
    tfidf_matrix = tfidf_vectorizer.fit_transform(dataframe[text_column])

    # Converte a matriz TF-IDF para um DataFrame para melhor legibilidade
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
    
    return tfidf_df


In [ ]:
# Calcula a matriz TF-IDF
tfidf_df = compute_tfidf(data, 'processed_text')

tfidf_df.head()

## Teste isolado

In [ ]:
# Teste: Verifica o valor TF-IDF para a palavra 'uber'
if 'uber' in tfidf_df.columns:
    uber_tfidf = tfidf_df['uber']
else:
    uber_tfidf = "A palavra 'uber' não está presente na matriz TF-IDF."

print(uber_tfidf)

# Naive Bayes

Nesta seção, utilizaremos o modelo Gaussian Naive Bayes para classificar as frases como positivas ou negativas para depois ser salvo na pasta [models](models).

In [ ]:
def train_and_save_model(file_path, model_save_path):
    """
    Treina e salva um modelo de classificação Naive Bayes em um arquivo.

    Input:
    file_path (str): O caminho para o arquivo CSV contendo os dados de treinamento.
    model_save_path (str): O caminho para salvar o arquivo do modelo treinado.

    Output:
    dict: Um dicionário contendo a acurácia do modelo, o relatório de classificação, e os conjuntos de teste.
    """
    # Lendo os dados do arquivo CSV
    data = pd.read_csv(file_path)

    # Verificando se a coluna 'processed_text' existe
    if 'processed_text' not in data.columns:
        raise ValueError("O arquivo CSV não contém a coluna 'processed_text'. Verifique o pré-processamento dos dados.")

    # Usando a coluna 'processed_text' para os dados de entrada
    X = data['processed_text']
    y = data['sentiment']

    # Convertendo os textos para uma matriz de características usando Bag of Words
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer()
    X_vectorized = vectorizer.fit_transform(X)

    # Dividindo os dados em conjuntos de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.3, random_state=42)

    # Modelo de classificação
    model = GaussianNB()

    # Treinando o modelo (usando toarray() porque o GaussianNB não suporta matrizes sparse)
    model.fit(X_train.toarray(), y_train)

    # Salvando o modelo treinado com pickle
    # Garantindo que o diretório de salvamento existe
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    
    with open(model_save_path, 'wb') as model_file:
        pickle.dump(model, model_file)

    # Predições no conjunto de teste
    y_pred = model.predict(X_test.toarray())

    # Avaliação do modelo
    accuracy = accuracy_score(y_test, y_pred)
    class_report = classification_report(y_test, y_pred, output_dict=True)

    return {
        "accuracy": accuracy,
        "classification_report": class_report,
        "X_test": X_test,  # Adiciona X_test ao retorno para uso posterior
        "y_test": y_test   # Adiciona y_test ao retorno para uso posterior
    }

file_path = './data/processed_text_data.csv'
model_save_path = 'models/naive_bayes_model.pkl'

results = train_and_save_model(file_path, model_save_path)
print(f"Modelo salvo em: {model_save_path}")
print(f"Acurácia: {results['accuracy']}")
print(f"Relatório de Classificação: \n{results['classification_report']}")

# XGBoost com W2V

A seguir é chamado o modelo XGBoost com os dados W2V, que treina o modelo e salva na pasta [models](models).

In [ ]:
def train_and_save_xgboost_model(file_path, model_save_path):
    """
    Treina um modelo XGBoost utilizando Word2Vec, exibe as métricas de avaliação e exporta o modelo treinado.

    Inputs: 
    file_path (str): Caminho para o arquivo CSV contendo os dados de treinamento.
    model_save_path (str): Caminho para o arquivo para salvar o modelo treinado.

    Outputs: dict: Um dicionário contendo as métricas de avaliação do modelo ou None se ocorrer um erro.
    """
    try:
        # Carregar os dados
        data = pd.read_csv(file_path)

        # Verificar a presença da coluna 'sentiment'
        if 'sentiment' not in data.columns:
            raise KeyError("'sentiment' não encontrado nas colunas do DataFrame")

        # Verificar a presença da coluna 'processed_text'
        if 'processed_text' not in data.columns:
            raise KeyError("'processed_text' não encontrado nas colunas do DataFrame")

        # Usar a coluna 'processed_text' para os dados de entrada
        X = data['processed_text']
        y = data['sentiment']

        # Re-rotular os rótulos para garantir que estão no intervalo [0, num_class)
        y = LabelEncoder().fit_transform(y)

        # Usar o CountVectorizer para transformar os textos em uma matriz de características
        from sklearn.feature_extraction.text import CountVectorizer
        vectorizer = CountVectorizer()
        X_vectorized = vectorizer.fit_transform(X)

        # Dividir os dados em conjuntos de treinamento e teste
        X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

        # Criar o DMatrix para o XGBoost
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)

        # Definir os parâmetros do XGBoost
        params = {
            'max_depth': 6,
            'eta': 0.3,
            'objective': 'multi:softmax',  # Usa softmax para classificação multi-classes
            'num_class': len(set(y))  # Número de classes no seu problema de classificação
        }

        # Treinar o modelo
        bst = xgb.train(params, dtrain, num_boost_round=100)

        # Fazer previsões
        y_pred = bst.predict(dtest)

        # Exibindo o relatório de classificação
        class_report = classification_report(y_test, y_pred)
        print("Classification Report:\n", class_report)

        # Calcular e visualizar a matriz de confusão
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
                    xticklabels=['Negativo', 'Neutro', 'Positivo'], 
                    yticklabels=['Negativo', 'Neutro', 'Positivo'])
        plt.title("Matriz de Confusão")
        plt.xlabel("Valor Previsto")
        plt.ylabel("Valor Real")
        plt.show()

        # Exportar o modelo usando pickle
        os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
        with open(model_save_path, 'wb') as model_file:
            pickle.dump(bst, model_file)

        return {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred, average='weighted'),
            'recall': recall_score(y_test, y_pred, average='weighted'),
            'f1_score': f1_score(y_test, y_pred, average='weighted'),
            'classification_report': class_report,  # Adiciona explicitamente o relatório de classificação ao dicionário
            'confusion_matrix': conf_matrix
        }

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None

file_path = 'data/processed_text_data.csv'
model_save_path = 'models/xgboost_model.pkl'

results = train_and_save_xgboost_model(file_path, model_save_path)
if results:
    print(f"Modelo salvo em: {model_save_path}")
    print(f"Acurácia: {results['accuracy']}")
    print(f"Relatório de Classificação: \n{results['classification_report']}")